# Baseline model for adult data

In this notebook we train a simple model on the adult data that can serve as a counterfactual for what would have happened if we hadn't made any kind of fairness intervention.

In [1]:
import joblib
from pathlib import Path

import pandas as pd
from fairlearn.metrics import (
    demographic_parity_difference,
    demographic_parity_ratio,
    equalized_odds_difference,
    equalized_odds_ratio,
)
from helpers.utils import bin_hours_per_week
from helpers.utils import (
    conditional_demographic_parity_difference,
    conditional_demographic_parity_ratio,
)
from helpers.utils import group_box_plots
from sklearn.neural_network import MLPClassifier  # noqa

Directory containing preprocessed data.

In [3]:
artifacts_dir = Path("artifacts")

Load the preprocessed data. Check out the preprocessing notebook for details on how this data was obtained.

In [4]:
data_dir = artifacts_dir / "data" / "adult"

train = pd.read_csv(data_dir / "processed" / "train.csv")
val = pd.read_csv(data_dir / "processed" / "val.csv")
test = pd.read_csv(data_dir / "processed" / "test.csv")

train_oh = pd.read_csv(data_dir / "processed" / "train-one-hot.csv")
val_oh = pd.read_csv(data_dir / "processed" / "val-one-hot.csv")
test_oh = pd.read_csv(data_dir / "processed" / "test-one-hot.csv")

In [10]:
train_oh.head()

,age,sex,capital_gain,capital_loss,hours_per_week,salary,workclass_federal_gov,workclass_local_gov,workclass_private,workclass_self_emp_inc,...,marital_status_divorced,marital_status_married_af_spouse,marital_status_married_civ_spouse,marital_status_married_spouse_absent,marital_status_never_married,marital_status_separated,marital_status_widowed,native_country_mexico,native_country_other,native_country_united_states
0,1.108936,1,-0.147741,-0.218133,2.416833,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
1,0.805386,1,-0.147741,-0.218133,2.416833,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2,-0.788255,1,-0.147741,-0.218133,-0.079269,0,0,0,1,0,...,0,0,1,0,0,0,0,0,1,0
3,1.640150,0,-0.147741,-0.218133,-0.079269,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,1
4,1.108936,1,-0.147741,-0.218133,-0.079269,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,1


## Training a model to predict salary

We will load a model from disk so that results are reproducible, but commented out here is the code we used to train the model.

In [11]:
import torch
from torch.utils.data import Dataset
class AdultDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.X = torch.tensor(df.drop(columns=["salary"]).values, dtype=torch.float32)
        self.y = torch.tensor(df["salary"].values, dtype=torch.float32)
        self.s = torch.tensor(df["sex"].values, dtype=torch.int64)  # sensitive attr

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx], self.s[idx]

In [12]:
from torch.utils.data import DataLoader

train_ds = AdultDataset(train_oh)
val_ds = AdultDataset(val_oh)
test_ds = AdultDataset(test_oh)

train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=128)
test_loader = DataLoader(test_ds, batch_size=128)

In [13]:
import torch.nn as nn

class FairMLP(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 100),
            nn.ReLU(),
            nn.Linear(100, 100),
            nn.ReLU(),
            nn.Linear(100, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x).squeeze()
    
def demographic_parity_loss(y_hat, s):
    group_0 = y_hat[s == 0]
    group_1 = y_hat[s == 1]
    if len(group_0) == 0 or len(group_1) == 0:
        return torch.tensor(0.0, device=y_hat.device)
    return (group_0.mean() - group_1.mean())**2

In [67]:
model = FairMLP(input_dim=train_oh.shape[1] - 1)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
bce_loss = nn.BCELoss()
lambda_dp = 0.7  # try different values

for epoch in range(10):
    model.train()
    for x, y, s in train_loader:
        optimizer.zero_grad()
        y_hat = model(x)
        loss_base = bce_loss(y_hat, y)
        loss_dp = demographic_parity_loss(y_hat, s)
        loss = loss_base + lambda_dp * loss_dp
        loss.backward()
        optimizer.step()

In [68]:
from sklearn.metrics import accuracy_score

model.eval()

X_test = torch.tensor(test_oh.drop(columns=["salary"]).values, dtype=torch.float32)
y_test = test_oh["salary"].values

with torch.no_grad():
    test_prob = model(X_test).numpy()  # shape (n_samples,)

test_pred = test_prob > 0.5  # binary threshold
test_accuracy = accuracy_score(y_test, test_pred)

print(f"Test accuracy: {test_accuracy * 100:.2f}%")

Test accuracy: 84.94%


In [47]:
model = MLPClassifier(hidden_layer_sizes=(100, 100), early_stopping=True)

model.fit(train_oh.drop(columns="salary"), train_oh.salary)

MLPClassifier(early_stopping=True, hidden_layer_sizes=(100, 100))

Load the pretrained model

In [ ]:
# model = joblib.load(artifacts_dir / "models" / "finance" / "baseline.pkl")

Model accuracy on test set

In [5]:
test_prob = model.predict_proba(test_oh.drop(columns="salary"))[:, 1]
test_pred = test_prob > 0.5
test_accuracy = model.score(test_oh.drop(columns="salary"), test_oh.salary)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")

Test accuracy: 84.84%


## Demographic parity

We measure demographic parity using both the difference and ratio of acceptance rates. These metrics are implemented for us in the `fairlearn` library.

We also compare the distribution of scores for each sex using box plots of scores.

In [69]:
dpd = demographic_parity_difference(
    test_oh.salary, test_pred, sensitive_features=test_oh.sex,
)
dpr = demographic_parity_ratio(
    test_oh.salary, test_pred, sensitive_features=test_oh.sex,
)

print(f"Demographic parity difference: {dpd:.3f}")
print(f"Demographic parity ratio: {dpr:.3f}")

Demographic parity difference: 0.122
Demographic parity ratio: 0.469


In [35]:
fig_dp_by_sex = group_box_plots(
    test_prob,
    test.sex.map(lambda x: "Male" if x else "Female"),
    title="Distribution of scores by sex",
    xlabel="Score",
)
fig_dp_by_sex

We calculate similar metrics and produce similar plots for race.

In [70]:
dpd = demographic_parity_difference(
    test_oh.salary, test_pred, sensitive_features=test.race,
)
dpr = demographic_parity_ratio(
    test_oh.salary, test_pred, sensitive_features=test.race,
)

print(f"Demographic parity difference: {dpd:.3f}")
print(f"Demographic parity ratio: {dpr:.3f}")

Demographic parity difference: 0.180
Demographic parity ratio: 0.251


In [37]:
race_names = {
    "amer_indian_eskimo": "American Indian / Eskimo",
    "asian_pac_islander": "Asian / Pacific Islander",
    "black": "Black",
    "other": "Other",
    "white": "White",
}

fig_dp_by_race = group_box_plots(
    test_prob,
    test.race.map(race_names),
    title="Distribution of scores by race",
    xlabel="Score",
)
fig_dp_by_race

## Conditional demographic parity

Distribution by sex and hours worked per week.

In [53]:
test_hpw_enum = test.hours_per_week.map(bin_hours_per_week)

cdpd = conditional_demographic_parity_difference(
    test_oh.salary, test_pred, test.sex, test_hpw_enum,
)
cdpr = conditional_demographic_parity_ratio(
    test_oh.salary, test_pred, test.sex, test_hpw_enum,
)

print(f"Conditional demographic parity difference: {cdpd:.3f}")
print(f"Conditional demographic parity ratio: {cdpr:.3f}")

Conditional demographic parity difference: 0.028
Conditional demographic parity ratio: 0.750


In [54]:
fig_cdp_by_sex = group_box_plots(
    test_prob,
    test.sex.map(lambda x: "Male" if x else "Female"),
    groups=test_hpw_enum,
    group_names=["0-30", "30-40", "40-50", "50+"],
    title="Distribution of scores by sex and hours worked per week",
    xlabel="Score",
    ylabel="Hours worked per week",
)
fig_cdp_by_sex

Distribution by race and hours worked per week.

In [55]:
cdpd = conditional_demographic_parity_difference(
    test_oh.salary, test_pred, test.race, test_hpw_enum,
)
cdpr = conditional_demographic_parity_ratio(
    test_oh.salary, test_pred, test.race, test_hpw_enum,
)

print(f"Conditional demographic parity difference: {cdpd:.3f}")
print(f"Conditional demographic parity ratio: {cdpr:.3f}")

Conditional demographic parity difference: 0.218
Conditional demographic parity ratio: 0.077


In [56]:
fig_cdp_by_race = group_box_plots(
    test_prob,
    test.race.map(race_names),
    groups=test_hpw_enum,
    group_names=["0-30", "30-40", "40-50", "50+"],
    title="Distribution of scores by race and hours worked per week",
    xlabel="Score",
    ylabel="Hours worked per week",
)
fig_cdp_by_race

## Equalised odds

To assess equalised odds we compare scores across the outcome classes.

In [71]:
eod = equalized_odds_difference(
    test_oh.salary, test_pred, sensitive_features=test.sex,
)
eor = equalized_odds_ratio(
    test_oh.salary, test_pred, sensitive_features=test.sex,
)

print(f"Equalised odds difference: {eod:.3f}")
print(f"Equalised odds ratio: {eor:.3f}")

Equalised odds difference: 0.067
Equalised odds ratio: 0.519


In [58]:
fig_eo_by_sex = group_box_plots(
    test_prob,
    test.sex.map(lambda x: "Male" if x else "Female"),
    groups=test.salary,
    group_names=["Low earner", "High earner"],
    title="Distribution of scores by sex for high and low earners",
    xlabel="Score",
    ylabel="High / low earner",
)
fig_eo_by_sex

We do the same, comparing races.

In [72]:
eod = equalized_odds_difference(
    test_oh.salary, test_pred, sensitive_features=test.race,
)
eor = equalized_odds_ratio(
    test_oh.salary, test_pred, sensitive_features=test.race,
)

print(f"Equalised odds difference: {eod:.3f}")
print(f"Equalised odds ratio: {eor:.3f}")

Equalised odds difference: 0.328
Equalised odds ratio: 0.255


In [60]:
fig_eo_by_race = group_box_plots(
    test_prob,
    test.race.map(race_names),
    groups=test.salary,
    group_names=["Low earner", "High earner"],
    title="Distribution of scores by race for high and low earners",
    xlabel="Score",
    ylabel="High / low earner",
)
fig_eo_by_race